**<span style="color: #808080;">In The Name of God</span>**

---

**<span style="color: #808080;">Erfan Falahati</span>**

**<span style="color: #808080;">SID:</span> <span style="color: #808080;">810102491</span>**

# **<span style="color: coral;">Computer Assignment #3: Machine Learning</span>**


# **<span style="color: #20B2AA;">Data Exploration in Tableau</span>**

# **<span style="color: #20B2AA;">Preprocessing and Feauture Engineering</span>**

In [132]:
DATASET_PATH = 'content/Grades.csv'

In [133]:
import pandas as pd

df = pd.read_csv(DATASET_PATH)

## **First Look Up in Data** 

In [134]:
df.describe()

,age,motherEducation,fatherEducation,travelTime,studyTime,failures,freeTime,goOut,Dalc,Walc,absences,EPSGrade,DSGrade,finalGrade
count,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000
mean,16.705290,2.748111,2.521411,1.450882,2.032746,0.335013,3.234257,3.108312,1.478589,2.289673,5.695214,10.881612,10.687657,10.377834
std,1.279716,1.097301,1.090672,0.696840,0.838731,0.742714,0.996476,1.112762,0.889143,1.286770,7.988015,3.333120,3.770322,4.604928
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,3.000000,2.000000,1.000000,1.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,3.000000,3.000000,1.000000,2.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,4.000000,4.000000,2.000000,3.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [135]:
from IPython.display import Markdown, display
# df.isnull().sum()

display(Markdown("### 📁 check for NaN values"))
display(df.isnull().sum().to_frame(name='Null Count').style.set_caption("Raw Bounding Box Info"))


### 📁 check for NaN values

,Null Count
university,0
sex,0
age,0
address,0
motherEducation,0
fatherEducation,0
motherJob,0
fatherJob,0
reason,0
travelTime,0


In [136]:
outliers = df[(df['absences'] > 30)]
outliers.sort_values(by='absences' ,ascending=False)


,university,sex,age,address,motherEducation,fatherEducation,motherJob,fatherJob,reason,travelTime,...,internet,romantic,freeTime,goOut,Dalc,Walc,absences,EPSGrade,DSGrade,finalGrade
276,PR,F,18,R,3,2,other,services,home,2,...,yes,yes,1,1,1,1,75,10,9,9
183,PR,F,17,U,3,3,other,other,reputation,1,...,yes,yes,3,3,2,3,56,9,9,8
74,PR,F,16,U,3,3,other,services,home,1,...,yes,no,3,3,2,4,54,11,12,11
315,PR,F,19,R,2,3,other,other,reputation,1,...,yes,yes,1,2,1,1,40,13,11,11
307,PR,M,19,U,4,4,teacher,services,reputation,2,...,yes,yes,3,4,1,1,38,8,9,8


In [137]:
df_v1 = df[df['absences'] <= 30]

In [172]:
other_counts_mother_job = df_v1[df_v1['fatherJob'] == 'other']['fatherJob'].count() / df_v1.shape[0]
other_counts_fatherjob = df_v1[df_v1['motherJob'] == 'other']['motherJob'].count() / df_v1.shape[0]
other_counts_reason = df_v1[df_v1['reason'] == 'other']['reason'].count() / df_v1.shape[0]


print("other counts for columns:")
print(f"motherJob others percentage: % {other_counts_mother_job}")
print(f"fatherJob others percentage: % {other_counts_fatherjob}")
print(f"reason others percentage: % {other_counts_reason}")


other counts for columns:
motherJob others percentage: % 0.548469387755102
fatherJob others percentage: % 0.3520408163265306
reason others percentage: % 0.09438775510204081


In [139]:
df_v2 = df_v1.drop(columns=['fatherJob', 'motherJob'])

### **Predict the reason** 

In [171]:
df_v2['reason'] = df_v2['reason'].mask((df['reason'] == 'other') & (df_v2['travelTime'] == 1))
df_v2[df]

C:\Users\Erfan\AppData\Local\Temp\ipykernel_9252\4214788734.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  others_df[df['travelTime']  == 1]


,university,sex,age,address,motherEducation,fatherEducation,reason,travelTime,studyTime,failures,...,internet,romantic,freeTime,goOut,Dalc,Walc,absences,EPSGrade,DSGrade,finalGrade
35,PR,F,15,U,2,3,other,2,1,0,...,0,0,5,1,1,1,0,8,7,6
52,PR,M,15,U,4,2,other,2,1,1,...,0,0,5,5,3,4,6,11,11,10
192,PR,M,17,U,1,2,other,2,2,0,...,1,0,4,4,4,5,12,7,8,8
311,PR,F,19,U,2,1,other,3,2,0,...,1,1,4,1,1,1,20,14,12,13
356,CM,F,17,R,4,4,other,2,2,0,...,1,0,3,3,1,2,4,12,13,13
361,CM,M,18,R,1,1,other,2,2,1,...,0,0,4,3,2,3,2,13,12,12
362,CM,F,18,U,3,3,other,2,2,0,...,1,1,3,2,1,3,0,11,11,10
366,CM,M,18,U,4,4,other,2,3,0,...,1,1,2,2,2,2,0,13,13,13
369,CM,F,18,R,4,4,other,3,2,0,...,1,1,2,2,4,2,10,14,12,11
371,CM,M,18,R,1,2,other,3,1,0,...,1,1,3,3,2,3,3,14,12,12


### **Map (yes,no) columns to (0,1)** 

In [140]:
df_v3 = df_v2
for column in ['paid', 'higher', 'internet', 'romantic', 'universitySupport']:
    df_v3[column] = df_v2[column].replace(['yes', 'no'], [1, 0])

print("✅")

✅


C:\Users\Erfan\AppData\Local\Temp\ipykernel_9252\1063575527.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_v3[column] = df_v2[column].replace(['yes', 'no'], [1, 0] )
